# Parameter Identification Technique (PIT) on a linear Nomoto model

# Purpose
Show the general workflow by applying PIT regression on a very simple model

# Methodology
* Load time series from a ZigZag test
* Determine yaw rate and acceleration (compare with accelerometers from test).
* Find the best parameter values in the Nomoto model using OLS linear regression.

# Setup

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl




## Load test

In [ ]:
df_runs = mdl.runs()

In [ ]:
id=22773
df, units, meta_data = mdl.load(id=id)

In [ ]:
meta_data.dropna()

In [ ]:
df.head()

In [ ]:
units = mdl.load_units()

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

## Yaw rate

In [ ]:
px.line(data_frame=df, y=['psi'], width=1400, height=300,)

In [ ]:
t = df.index.total_seconds()
df['r_numerical'] = np.gradient(df['psi'],t)
df['r1d_numerical'] = np.gradient(df['r_numerical'],t)


## Yaw rate from Kalman filter

In [ ]:
from vessel_manoeuvring_models.data import kalman_filter

In [ ]:
df = kalman_filter.yaw(df=df)

display(px.line(data_frame=df, y=['psi','psi_filtered'], width=1400, height=300,))
display(px.line(data_frame=df, y=['r_numerical','r'], width=1400, height=300,))
display(px.line(data_frame=df, y=['r1d'], width=1400, height=300,))

In [ ]:
display(px.line(data_frame=df.resample('2S').mean(), y=['Hull/Acc/X1','Hull/Acc/Y1','Hull/Acc/Y2','Hull/Acc/Z1','Hull/Acc/Z2','Hull/Acc/Z3'], width=1400, height=300,))

# First order Nomoto model for ship maneuvering dynamics
The first order Nomoto model can be written as:

In [ ]:
K, delta, T_1, T_2 = sp.symbols('K delta T_1 T_2')
r = dynamicsymbols('r')

eq_nomoto = sp.Eq(-K*delta,
                 r + T_1*r1d + T_2*r1d.diff())
Math(vlatex(eq_nomoto))

where $r$ is yaw rate with its time derivatives and $\delta$ is the rudder angle. $K$, $T_{1}$
 and $T_{1}$ are the coefficients describing the hydrodynamics of the ship.
 
For slow manoeuvres this equation can be further simplified by removing the $\ddot{r}$ term:

In [ ]:
eq_nomoto_simple = eq_nomoto.subs(r1d.diff(),0)
Math(vlatex(eq_nomoto_simple))

In [ ]:
eq_r1 = sp.Eq(r1d,
             sp.solve(eq_nomoto_simple, r1d)[0])
r1d_lambda = lambdify(sp.solve(eq_nomoto_simple, r1d)[0])
Math(vlatex(eq_r1))


In [ ]:
def nomoto(t,states,df_control,params):
    """
    Simulation model for heading and yaw rate using Nomoto with only K and T_1
    """
    # states:
    psi = states[0]
    r = states[1]
    
    index = df_control.index.get_loc(t, method='nearest')
    delta = float(df_control['delta'].iloc[index])
    r1d_ = r1d_lambda(K=params['K'], T_1=params['T_1'], delta=delta, r=r)
    d_states_dt = np.array([r,r1d_])
    return d_states_dt

## Simulate a pull out manoeuvre
Simulating a simple pull out manoeuvre:

In [ ]:
params = {
    'K' :  0.1,
    'T_1' : 10,
}

states_0 = np.array([df.iloc[0]['psi'],df.iloc[0]['r']])

df_control = df.copy()
df_control.index = df_control.index.total_seconds()
t = df_control.index

sol = solve_ivp(fun = nomoto,t_span=[t[0],t[-1]],t_eval=t, y0 = states_0, args=(df_control,params))
psi = sol.y[0,:]
r = sol.y[1,:]

fig,ax=plt.subplots()
df_control['psi_deg'] = np.rad2deg(df_control['psi'])
df_control.plot(y='psi_deg', label='model test', ax=ax)
ax.plot(t,np.rad2deg(psi), label='simulation');
ax.grid(True)
ax.set_title('Resimulation with Nomoto model')
ax.set_ylabel('$\psi$ [deg]')
ax.set_xlabel('time [s]');
ax.legend()


## Regress nomoto parameters

In [ ]:
eq_r1

In [ ]:
import sympy.physics.mechanics as me

me.find_dynamicsymbols(eq_r1)

In [ ]:
eq_r1.rhs

In [ ]:
y = df_control['r1d']
X = df_control[['delta','r']] 

#y = df_control['r1d_numerical']
#X = df_control[['delta','r_numerical']] 
#X.rename(columns={'r_numerical': 'r'}, inplace=True)

model = sm.OLS(y,X)
results = model.fit()
results.summary()

In [ ]:
r1d_pred = results.predict(X)
X_ = X.iloc[::100, :]
prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)

fig,ax=plt.subplots()
ax.plot(t,y, label=r'$\dot{r}$')
ax.plot(t,r1d_pred, label=r'$\dot{r}_{pred}$')

ax.plot(X_.index,iv_l, 'k--')
ax.plot(X_.index,iv_u, 'k--')
ax.legend()

In [ ]:
params['T_1'] = -1/results.params['r']
params['K'] = -results.params['delta']*params['T_1']

In [ ]:
states_0 = np.array([df.iloc[0]['psi'],df.iloc[0]['r']])

df_control = df.copy()

df_control.index = df_control.index.total_seconds()
t = df_control.index

sol = solve_ivp(fun = nomoto,t_span=[t[0],t[-1]],t_eval=t, y0 = states_0, args=(df_control,params))
psi = sol.y[0,:]
r = sol.y[1,:]

fig,ax=plt.subplots()
df_control['psi_deg'] = np.rad2deg(df_control['psi'])
df_control.plot(y='psi_deg', label='model test', ax=ax)

df_control['-delta_deg'] = -np.rad2deg(df_control['delta'])
df_control.plot(y='-delta_deg', label=r'$-\delta$', ax=ax)

ax.plot(t,np.rad2deg(psi), label='simulation');
ax.grid(True)
ax.set_title('Resimulation with Nomoto model')
ax.set_ylabel('$\psi$ [deg]')
ax.set_xlabel('time [s]');
ax.legend();


In [ ]:
params